# setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  1.0.2
TF version:  2.7.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


# Get the data

In [2]:
import tensorflow_datasets as tfds
# You can find lists of datasets at https://www.tensorflow.org/datasets/catalog/overview
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
# Information of dataset split (train/validation/test)
info.splits
# Information of train dataset
info.splits["train"]
# Name of each class
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes
print(n_classes)
dataset_size = info.splits["train"].num_examples
# Split dataset using split method
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)
from functools import partial

# Define preprocess functions
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

# Select central crop or randomized crop
def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    print(resized_image.shape)
    final_image = keras.applications.mobilenet.preprocess_input(resized_image)
    return final_image, label

# Define Train, validation and test datasets
batch_size = 16
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

5
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


In [4]:
train_set_raw

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [15]:
from functools import partial
def preprocess(image,label ):
    resized_image = tf.image.resize(image, [224, 224])
    resized_image2=tf.concat([resized_image,resized_image],axis=3)
    resized_image2=tf.concat([resized_image2,resized_image],axis=3)
    print(resized_image2.shape)
    final_image = keras.applications.mobilenet.preprocess_input(resized_image2)
    return final_image,label

# Define Train, validation and test datasets
batch_size = 16
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess)).batch(batch_size).prefetch(1)

(55000, 224, 224, 3)


In [22]:
train_set_raw

<PrefetchDataset element_spec=(TensorSpec(shape=(55000, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(55000,), dtype=tf.uint8, name=None))>

# SMV2L

In [17]:
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [19]:
smv2_l=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[224,224,3]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])

In [10]:
smv2_l.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 112, 112, 16)      1216      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 56, 56, 16)        6688      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 28, 28, 32)        38656     
_________________________________________________________________
IRB4 (Inveted_Residual_Block (None, 14, 14, 64)        134656    
_________________________________________________________________
IRB5 (Inveted_Residual_Block (None, 7, 7, 128)         2

In [6]:
smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

In [8]:
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam2.h5",save_best_only=True)
history=smv2_l.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10,callbacks=[check_cb])

Epoch 1/10
172/172 [==============================] - 17s 96ms/step - loss: 1.5512 - accuracy: 0.3779 - val_loss: 1.7621 - val_accuracy: 0.1654
Epoch 2/10
172/172 [==============================] - 17s 96ms/step - loss: 1.3904 - accuracy: 0.4415 - val_loss: 1.7141 - val_accuracy: 0.2390
Epoch 3/10
172/172 [==============================] - 16s 92ms/step - loss: 1.2982 - accuracy: 0.4964 - val_loss: 2.8134 - val_accuracy: 0.2390
Epoch 4/10
172/172 [==============================] - 16s 93ms/step - loss: 1.2171 - accuracy: 0.5302 - val_loss: 1.2407 - val_accuracy: 0.5092
Epoch 5/10
172/172 [==============================] - 16s 96ms/step - loss: 1.1591 - accuracy: 0.5425 - val_loss: 1.2270 - val_accuracy: 0.5772
Epoch 6/10
172/172 [==============================] - 16s 93ms/step - loss: 1.0863 - accuracy: 0.5879 - val_loss: 1.1859 - val_accuracy: 0.5423
Epoch 7/10
172/172 [==============================] - 16s 96ms/step - loss: 1.0530 - accuracy: 0.5981 - val_loss: 0.9143 - val_accuracy:

# SMV2P

In [18]:
smv2_p=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[224,224,3]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=128,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])

In [19]:
smv2_p.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 224, 224, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 112, 112, 16)     1216      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 56, 56, 16)       6688      
 k)                                                              
                                                                 
 IRB3 (Inveted_Residual_Bloc  (None, 28, 28, 64)       126400    
 k)                                                     

In [20]:
smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [21]:

check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam.h5",save_best_only=True)
history=smv2_p.fit(train_set,
                    steps_per_epoch=int( dataset_size / batch_size),
                    epochs=5)

Epoch 1/5


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of la

In [31]:
model=keras.models.load_model("smv2l_nadam.h5",custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})
model.evaluate(test_set)

23/23 [==============================] - 0s 18ms/step - loss: 0.4342 - accuracy: 0.2918


[0.4341720938682556, 0.29178759455680847]

In [30]:
model2=keras.models.load_model("my_mobilenetv2_1.h5",
                               custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})
model2.evaluate(test_set)

23/23 [==============================] - 1s 23ms/step - loss: 1.1099 - accuracy: 0.3706


[1.1099331378936768, 0.37057220935821533]

In [36]:
model3=keras.models.load_model("smv2p_nadam.h5",
                               custom_objects={"Inveted_Residual_Block" : Inveted_Residual_Block})
model3.evaluate(test_set)

23/23 [==============================] - 0s 19ms/step - loss: 0.5485 - accuracy: 0.2779


[0.5485105514526367, 0.2779291570186615]

In [37]:
model.evaluate(valid_set)

35/35 [==============================] - 1s 19ms/step - loss: 0.4999 - accuracy: 0.2585


[0.4998670816421509, 0.25853991508483887]

In [38]:
model3.evaluate(valid_set)

35/35 [==============================] - 1s 24ms/step - loss: 0.5297 - accuracy: 0.2468


[0.5296900868415833, 0.24682395160198212]

In [ ]:
dataset, info = tfds.load("emnist", as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]